In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
from IPython.display import clear_output

------
<a id="indice"></a>

# Índice

1. **[Coleta](#coleta)**
    * [Twitter Archive](#coleta:twitter-archive)
    * [Image Prediction](#coleta:image-prediction)
    * [Twitter API](#coleta:twitter-api)
    
* **[Avaliação](#avaliacao)**
    * [Twitter Archive](#avaliacao:twitter-archive)
    * [Image Prediction](#avaliacao:image-prediction)
    * [Twitter API](#avaliacao:twitter-api)


------
<a id="coleta"></a>

# Coleta

<a id="coleta:twitter-archive"></a>

## Coleta: Twitter archive

In [2]:
df_twitter_arc = pd.read_csv('data/twitter-archive-enhanced.csv')
df_twitter_arc.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
787,774639387460112384,NaN,NaN,2016-09-10 16:03:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sprinkles. He's trapped in light jail....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/774639387...,10,10,Sprinkles,None,None,None,None
1659,683098815881154561,NaN,NaN,2016-01-02 01:33:43 +0000,"<a href=""http://twitter.com/download/iphone"" r...",aahhhhkslaldhwnxmzbbs 12/10 for being da smoos...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/683098815...,12,10,None,None,None,None,None


<a id="coleta:image-prediction"></a>

## Coleta: Image prediction

In [3]:
df_prediction = None

r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

if r.status_code is 200:
    df_prediction = pd.read_csv(pd.compat.StringIO(r.text), sep='\t')    
else:    
    print('ERROR: Image prediction request returned {status_code} status code.'.format(status_code = r.status_code))

In [4]:
df_prediction.sample(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1064,715360349751484417,https://pbs.twimg.com/media/Ce14cOvWwAAcFJH.jpg,1,nail,0.855552,False,screw,0.073277,False,padlock,0.023970,False
328,672068090318987265,https://pbs.twimg.com/media/CVOqW8eUkAESTHj.jpg,1,pug,0.863385,True,shopping_cart,0.125746,False,Border_terrier,0.002972,True


<a id="coleta:twitter-api"></a>

## Coleta: Twitter API

In [5]:
# Twitter APP Config
with open('twitter_config.json', 'r', encoding='utf-8') as file:
    app_config = json.load(file)

In [6]:
# Twitter API settings
api_key = app_config['api_key']
api_secret = app_config['api_secret']
access_token = app_config['access_token']
access_secret = app_config['access_secret']

In [7]:
# Connect to Twitter API
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [8]:
json_path = 'data/tweet_json.txt'
error_log = 'data/tweet_error.log'

# Save tweets from Twitter API
if(not os.path.isfile(json_path)):
    print('Aguarde. Dado o volume de requisições, a coleta de tweets pode demorar alguns minutos.')
    tweet_count = 0
    for tweet_id in df_twitter_arc.tweet_id:
        print('{percent}%'.format(percent=int((++tweet_count/df_twitter_arc.tweet_id.size)*100)))
        try:
            status = api.get_status(tweet_id)
            with open(json_path, 'a', newline='\n') as file:
                file.write(f'{json.dumps(status._json)}\n')

        except Exception as err:
            with open(error_log, 'a', newline='\n') as log:
                log.write(f'{str(tweet_id)}: {str(err.args[0])}\n')
            print(f'{str(tweet_id)}: {str(err.args[0])}')
        
        clear_output(wait=True)
else:
    print('Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.')
        

Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.


In [9]:
# Convert JSON data into dict list
tweets = []

with open(json_path, 'r') as file:
    for line in file:
        try:
            tweet = json.loads(line)
            
            if(tweet.get('entities', False)):
                if(tweet['entities'].get('media', False) and tweet['entities']['media'][0].get('media_url', False)):
                    tweets.append({
                        'id': tweet['id'],
                        'created_at': tweet['created_at'],
                        'in_reply_to_status_id': tweet['in_reply_to_status_id'],
                        'in_reply_to_status_id': tweet['in_reply_to_status_id'],
                        'is_quote_status': tweet['is_quote_status'],
                        'retweet_count': tweet['retweet_count'],
                        'favorite_count': tweet['favorite_count'],
                        'media_url': tweet['entities']['media'][0]['media_url'],
                        'retweeted': tweet['retweeted'],
                        'favorited': tweet['favorited']
                    })
            
            
        except Exception as e:
            print(e) 
            
        

In [10]:
# Criar dataframe de Tweets consultados na API
columns = tweets[0].keys()
df_tweets_api = pd.DataFrame(tweets, columns = columns)

------
<a id="avaliacao"></a>

# Avaliação

<a id="avaliacao:twitter-archive"></a>

## Avaliação: Twitter data archive

In [11]:
 df_twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1704,680889648562991104,NaN,NaN,2015-12-26 23:15:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cheesy. It's her birthday. She's patie...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680889648...,9,10,Cheesy,None,None,None,None
1807,676916996760600576,NaN,NaN,2015-12-16 00:09:23 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Super speedy pupper. Does not go gentle into t...,NaN,NaN,NaN,https://vine.co/v/imJ0BdZOJTw,10,10,None,None,None,pupper,None
750,779123168116150273,NaN,NaN,2016-09-23 01:00:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Reggie. He hugs everyone he meets. 12/...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/779123168...,12,10,Reggie,None,None,None,None
1915,674318007229923329,NaN,NaN,2015-12-08 20:01:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Lennon. He's in quite the predicament....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674318007...,8,10,Lennon,None,None,pupper,None
834,768473857036525572,NaN,NaN,2016-08-24 15:43:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Chevy. He had a late breakfast and now ha...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/768473857...,11,10,Chevy,None,None,None,None


In [12]:
df_twitter_arc.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [13]:
df_twitter_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<a id="twitter-arc:info"></a>
* Dados incompletos, contém apenas 2356 registros dos 5000 anunciados
* `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` e `retweeted_status_user_id` em `float`
* `timestamp` e `retweeted_status_timestamp` em `string`

In [14]:
# Verificar se há IDs duplicados
df_twitter_arc.tweet_id.duplicated().value_counts()

False    2356
Name: tweet_id, dtype: int64

<a id="twitter-arc:source"></a>

In [15]:
# Visualizar exemplos de dados da coluna `source`
df_twitter_arc.source.sample(5)

568     <a href="http://twitter.com/download/iphone" r...
524     <a href="http://twitter.com/download/iphone" r...
301     <a href="http://twitter.com/download/iphone" r...
2116    <a href="http://twitter.com/download/iphone" r...
2071    <a href="http://twitter.com/download/iphone" r...
Name: source, dtype: object

Variável `source` sem valor para a unidade de observação

<a id="twitter-arc:name"></a>

In [16]:
# Visualizar exemplos da coluna `name`
df_twitter_arc.name.sample(10)

951       Lola
2085     Mason
190       None
767      Arnie
808      Davey
923       None
1478      Phil
832     Oakley
2058      Lola
157       None
Name: name, dtype: object

In [17]:
df_twitter_arc[df_twitter_arc.name == 'None'].name.count()

745

* Nomes (`name`) preenchidos com valor literal igual a `'None'`, vazio

<a id="twitter-arc:category"></id>

In [18]:
# Visualizar exemplo de dados a partir da coluna `doggo`
df_twitter_arc.loc[:,'doggo':].sample(10)

,doggo,floofer,pupper,puppo
1932,None,None,None,None
1722,None,None,None,None
1876,None,None,None,None
649,None,None,None,None
1172,None,None,None,None
2159,None,None,None,None
598,None,None,None,None
2281,None,None,None,None
1348,None,None,pupper,None
311,None,None,None,None


Uma variável em várias colunas

In [19]:
# Verificar padrão no preenchimento das classificações
df_twitter_arc.loc[:,'doggo':].nunique()

doggo      2
floofer    2
pupper     2
puppo      2
dtype: int64

<a id="twitter-arc:expanded_urls"></a>

Registros de `expanded_urls` com URLs inválidas e não existentes

In [20]:
# URLs sem conteúdo
df_twitter_arc.expanded_urls.sample(10)

738     https://twitter.com/dog_rates/status/780800785...
176     https://twitter.com/dog_rates/status/857746408...
690     https://twitter.com/dog_rates/status/787397959...
1834                        https://vine.co/v/iZXg7VpeDAv
558     https://twitter.com/dog_rates/status/767754930...
420     https://twitter.com/dog_rates/status/786233965...
1443                        https://vine.co/v/i1wrljBUjAu
39      https://twitter.com/dog_rates/status/884876753...
983     https://twitter.com/dog_rates/status/749395845...
2157    https://twitter.com/dog_rates/status/669583744...
Name: expanded_urls, dtype: object

In [21]:
df_twitter_arc.expanded_urls.isnull().value_counts()

False    2297
True       59
Name: expanded_urls, dtype: int64

------
<a id="avaliacao:image-prediction"></a>

## Avaliação: Image prediction

In [22]:
df_prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [23]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [24]:
df_prediction.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1752,824325613288833024,https://pbs.twimg.com/media/C3CXxaoWQAAiLuC.jpg,1,Pembroke,0.990793,True,Cardigan,0.008919,True,basenji,0.000262,True
260,670783437142401025,https://pbs.twimg.com/media/CU8Z-OxXAAA-sd2.jpg,1,lacewing,0.381955,False,sulphur_butterfly,0.106810,False,leafhopper,0.068347,False
174,669015743032369152,https://pbs.twimg.com/media/CUjSRNCXAAQ6Y_8.jpg,1,comic_book,0.275927,False,bib,0.173516,False,jersey,0.073911,False
297,671390180817915904,https://pbs.twimg.com/media/CVFBzpXVEAAHIOv.jpg,1,zebra,0.997673,False,tiger,0.000837,False,prairie_chicken,0.000575,False
1958,865359393868664832,https://pbs.twimg.com/media/DAJfxqGVoAAnvQt.jpg,2,Chesapeake_Bay_retriever,0.832435,True,Labrador_retriever,0.163551,True,Weimaraner,0.002770,True
966,706291001778950144,https://pbs.twimg.com/media/Cc0_2tXXEAA2iTY.jpg,1,Border_terrier,0.587101,True,bull_mastiff,0.164087,True,Staffordshire_bullterrier,0.105011,True
2059,889278841981685760,https://pbs.twimg.com/ext_tw_video_thumb/88927...,1,whippet,0.626152,True,borzoi,0.194742,True,Saluki,0.027351,True
1001,708738143638450176,https://pbs.twimg.com/media/CdXxlFPWwAABaOv.jpg,1,Pomeranian,0.933457,True,Samoyed,0.057221,True,West_Highland_white_terrier,0.000904,True
249,670679630144274432,https://pbs.twimg.com/media/CU67jGSUkAAk_1Y.jpg,1,Ibizan_hound,0.342734,True,Brittany_spaniel,0.229065,True,Chihuahua,0.104029,True
1886,847962785489326080,https://pbs.twimg.com/media/C8SRpHNUIAARB3j.jpg,1,sea_lion,0.882654,False,mink,0.066880,False,otter,0.025679,False


<a id="image-prediction:p-values"></a>

Valores não padronizados para as variáveis `p1`, `p2` e `p3`

In [25]:
# Visualização de exemplos de valores nas colunas `p1`, `p2` e `p3`
df_prediction.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
1756,Eskimo_dog,Siberian_husky,malamute
86,Italian_greyhound,standard_poodle,cocker_spaniel
223,Shetland_sheepdog,Cardigan,guinea_pig
327,chow,hamster,Pomeranian
1537,Bernese_mountain_dog,EntleBucher,Appenzeller


<a id="image-prediction:duplicated-urls"></a>

Verificar a duplicidade de Tweet ID (`tweet_id`) e Imagens (`jpg_url`)

In [26]:
# Verificar a duplicidade de IDs
df_prediction.tweet_id.nunique()

2075

In [27]:
# Verificar a duplicidade de Imagens analizadas
df_prediction.jpg_url.nunique()

2009

In [28]:
df_prediction.jpg_url.duplicated().value_counts()

False    2009
True       66
Name: jpg_url, dtype: int64

Há 66 imagens duplicadas

------
<a id="avaliacao:twitter-api"></a>

## Avaliação: Twitter API Requests

In [29]:
df_tweets_api.describe()

,id,in_reply_to_status_id,retweet_count,favorite_count
count,1.820000e+03,2.200000e+01,1820.000000,1820.000000
mean,7.237270e+17,6.992047e+17,2512.144505,6820.872527
std,5.777841e+16,4.409222e+16,4891.076169,11900.984651
min,6.660209e+17,6.671522e+17,11.000000,0.000000
25%,6.747671e+17,6.747625e+17,536.000000,1363.500000
50%,7.008223e+17,6.799651e+17,1118.000000,3153.000000
75%,7.617466e+17,7.032024e+17,2526.250000,7463.250000
max,8.924206e+17,8.558181e+17,82905.000000,163034.000000


<a id="tweets-api:info"></a>

In [30]:
df_tweets_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 9 columns):
id                       1820 non-null int64
created_at               1820 non-null object
in_reply_to_status_id    22 non-null float64
is_quote_status          1820 non-null bool
retweet_count            1820 non-null int64
favorite_count           1820 non-null int64
media_url                1820 non-null object
retweeted                1820 non-null bool
favorited                1820 non-null bool
dtypes: bool(3), float64(1), int64(3), object(2)
memory usage: 90.7+ KB


In [31]:
df_tweets_api.sample(5)

,id,created_at,in_reply_to_status_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
1591,670385711116361728,Fri Nov 27 23:36:23 +0000 2015,NaN,False,214,563,http://pbs.twimg.com/media/CU2wPyWWUAAb1MJ.jpg,False,False
842,707297311098011648,Tue Mar 08 20:09:54 +0000 2016,NaN,False,831,2877,http://pbs.twimg.com/media/CdDTJLMW4AEST--.jpg,False,False
1288,676821958043033607,Tue Dec 15 17:51:44 +0000 2015,NaN,False,16328,23702,http://pbs.twimg.com/media/CWSN-vbWcAATV7y.jpg,False,False
648,737310737551491075,Mon May 30 15:52:33 +0000 2016,NaN,False,7841,15255,http://pbs.twimg.com/ext_tw_video_thumb/737310...,False,False
1073,687704180304273409,Thu Jan 14 18:33:48 +0000 2016,NaN,False,884,2542,http://pbs.twimg.com/media/CYs3TKzUAAAF9A2.jpg,False,False


* `created_at` como `string`
* `in_reply_to_status_id` como `Float`

<a id="tweets-api:duplicated"></a>
Verificar a duplicidade de valores

In [32]:
# Verificar se há IDs duplicados
df_tweets_api.id.duplicated().value_counts()

False    1820
Name: id, dtype: int64

In [33]:
# Verificar se há imagens duplicadas
df_tweets_api.media_url.duplicated().value_counts()

False    1759
True       61
Name: media_url, dtype: int64

Há tweets referenciando a mesma imagem

------
<a id="assses-notes"></a>

## Notas de avaliação
### Qualidade

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, contém apenas **2356** registros dos **5000** anunciados](#twitter-arc:info)
* [Coluna `source` não acrescenta valor à unidade de obsevação](#twitter-arc:source)
* [Nomes (`name`) preenchidos com valor literal igual a `'None'`, vazio](#twitter-arc:name)
* [Coluna `expanded_urls` com linhas não preenchidas e sem valor para a unidade em observação](#twitter-arc:expanded_urls)


#### `df_prediction` - Image prediction
1. [Previsões [`p1`, `p2`, `p3`] com nomes não padronizados](#image-prediction:p-values)
* [Previsões repetidas sobre a mesma imagem](#image-prediction:duplicated-urls)


#### `df_tweets_api` - Remaining data from Tweets from Twitter API
1. [Data de criação (`created_at`) como `string`](#tweets-api:info)
* [Valor do id na coluna `in_reply_to_status_id` como `float`](#tweets-api:info)
* [Diferentes Tweets referenciando a mesma imagem, duplicidade](#tweets-api:duplicated)


### Organização

#### `df_twitter_arc` - Twitter archive
1. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#twitter-arc:info)
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string`](#twitter-arc:info)
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#twitter-arc:category)


#### `df_prediction` - Image prediction
-- *Nada a declarar* --


#### `df_tweets_api` - Remaining data from Tweets from Twitter API
1. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#tweets-api:info)

------
<a id="limpeza"></a>

# Limpeza

<a id="assses-notes"></a>

## Limpeza: Definição 

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, de **5000**, apenas **2356** estão disponíveis](#twitter-arc:info)
    * OK - Nada a ser feito
* [Coluna `source` apresenta tags HTML](#twitter-arc:source)
* [Coluna `source` não acrescenta valor à unidade de observação](#twitter-arc:source)
    * As tags HTML poderiam ser removidas com a utilização de expressões regulares. O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida
* [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#twitter-arc:name)
    * Aplicar valor '' para nomes iguais a 'None'
* [Coluna `expanded_urls` com linhas não preenchidas](#twitter-arc:expanded_urls)
* [Endereços inválidos em `expanded_urls`](#twitter-arc:expanded_urls)
    * Alguns endereços retornaram erro 404 e outros não preenchidos. Utilizar apenas tweets com referências válidas na API (`df_tweets_api`)
* [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweet_status_id`, `retweet_status_user_id` estão em `float64`](#twitter-arc:info)
    * A coluna, `retweet_status_id` ou `retweet_status_user_id`, pode ser convertida para `Boolean` como auxílio a identificação e desclassificação dos mesmos, objetivo da análise de avaliar somente tweets originais e não retweets
    * A coluna de retweet não utilizada na conversão booleana pode ser descartada
    * As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#twitter-arc:info)
    * Converter valores para `datetime`
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#twitter-arc:category)
    * Reduzir a variável para uma coluna, `category`


#### `df_prediction` - Image prediction
1. [Previsões [`p1`, `p2`, `p3`] com nomes não padronizados
    * Converter os valores para 'lower case' e substituir '_' por espaços
* [Previsões repetidas sobre a mesma imagem](#image-prediction:duplicated-urls)
    * Descartar previsões repetidas


#### `df_tweets_api` - Remaining data from Tweets from Twitter API
1. [Data de criação (`created_at`) como `string`](#tweets-api:info)
    * Converter datas em string para o formato `datetime`
* [Valor do id na coluna `in_reply_to_status_id` e `in_reply_to_user_id` como `float`](#tweets-api:info)
    * Converter para Inteiro as colunas `in_reply_to_status_id` e `in_reply_to_user_id` e atribuir 0 as que estiverem sem valor


## Limpeza: Twitter archive

In [44]:
# Criar cópia do dataframe para a limpeza
df_arch_clean = df_twitter_arc.copy()

2. [Coluna `source` apresenta tags HTML](#twitter-arc:source)
* [Coluna `source` não acrescenta valor à unidade de observação](#twitter-arc:source)
    * As tags HTML poderiam ser removidas com a utilização de expressões regulares. O conteúdo apresenta um link para download do App Twitter e sem valor à unidade de observação, a coluna pode ser removida

In [45]:
# Remoção da coluna `source`
df_arch_clean.drop('source', axis='columns', inplace=True)

#### Teste: Remoção coluna `source`

In [46]:
# Conferir que a coluna `source` foi removida
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

4. [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#twitter-arc:name)
    * Aplicar valor '' para nomes iguais a 'None'

In [47]:
df_arch_clean[df_arch_clean.name == 'None'].name.count()

745

In [48]:
# Aplicar '' aos nomes iguais a 'None'
df_arch_clean.name = df_arch_clean.name.apply(lambda name: '' if name.lower() == 'none' else name)

#### Teste: Nomes não preenchidos

In [49]:
df_arch_clean[df_arch_clean.name == ''].name.count()

745

In [50]:
df_arch_clean.name.sample(10)

458            
1293           
1222     Travis
327       Gabby
483      Cooper
1620       Oreo
1383    Crouton
219       Riley
497            
1930      Kaiya
Name: name, dtype: object

5. [Coluna `expanded_urls` com linhas não preenchidas](#twitter-arc:expanded_urls)
* [Endereços inválidos em `expanded_urls`](#twitter-arc:expanded_urls)
    * Alguns endereços retornaram erro 404 e outros não preenchidos. A informação reflete a URL do tweet original, sem valor para o objeto de avaliação. (`df_tweets_api`)

In [52]:
df_arch_clean.drop('expanded_urls', axis='columns', inplace=True)

#### Teste: Remoção da coluna `expanded_urls`

In [54]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'rating_numerator', 'rating_denominator',
       'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

7. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64` e deveriam ser em `int64`](#twitter-arc:info)
    * Haja vista que estas mesmas colunas estão presentes, também, no dataframe resultante das requisições à API, elas serão tratadas na limpeza do segundo (`df_tweets_api`), 

In [ ]:
df_arch_clean[['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id']].astype(int)

## Limpeza: Image prediction 

# Armazenamento

# Relatórios

* Data wrangling efforts
* Analyses and visualizations